In [1]:
import os
from dotenv import load_dotenv

In [3]:
try:
    # Load environment variables
    load_dotenv()
    os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
    os.environ['LANGSMITH_API_KEY'] = os.getenv('LANGSMITH_API_KEY')
    os.environ['LANGSMITH_TRACING'] = "true"
    os.environ["LANGSMITH_PROJECT"] = "test translator"

except ImportError:
    pass

In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, model='gpt-4')

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage
messages = [
SystemMessage("Translate the following from English into Italian"),
HumanMessage("Hi")
]

model.invoke(messages)

AIMessage(content='Ciao', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 19, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f972e277-c1e8-4629-97e5-17578bdd0f4d-0')

In [6]:
for token in model.stream(messages):
    print(token.content, end="|")

|C|iao||

In [7]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {languaje}"

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        ('user', '{text}')
    ]
)

In [9]:
prompt = prompt_template.invoke({'languaje': 'Italian', 'text': 'Hi'})
prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Italian'), HumanMessage(content='Hi')])

In [11]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into Italian'),
 HumanMessage(content='Hi')]

In [12]:
response = model.invoke(prompt)
print(response.content)

Ciao


### Classify Text into Labels

In [13]:
model = ChatOpenAI(temperature=0, model='gpt-4')

In [15]:
from pydantic import BaseModel, Field

In [17]:
prompt = """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
tagging_prompt = ChatPromptTemplate.from_messages(
    [
        ('user', prompt)
    ]
)

class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggresiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    languaje: str = Field(description="The languaje the text is written in")

# Sttructured LLM
structured_llm = model.with_structured_output(Classification)

In [22]:
inp = "Soy la persona mas feliz del planeta tierra quiero abrazarlos a todos"
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)

response

{'sentiment': 'feliz', 'aggresiveness': 0, 'languaje': 'Spanish'}

In [25]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)
response

{'sentiment': 'angry', 'aggresiveness': 8, 'languaje': 'spanish'}

In [32]:
class Classification(BaseModel):
    sentiment: str = Field(
        description="The sentiment of the text", 
        enum=["happy", "neutral", "sad"]
    )
    aggressiveness: int = Field(
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        description="The languaje the text is written in",
        enum=["spanish", "english", "french", "german", "italian"]
    )

In [35]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)
llm = model.with_structured_output(Classification)

In [36]:
init = 'Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!'

prompt = tagging_prompt.invoke({'input': init})

response = llm.invoke(prompt)
response

{'sentiment': 'contento', 'aggressiveness': 0, 'language': 'Spanish'}

### Build and extraction Chain

In [37]:
from typing import Optional

class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )

In [38]:
from langchain_core.prompts import MessagesPlaceholder

# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

In [51]:
llm = ChatOpenAI(temperature=0, model='gpt-4')
structured_model = llm.with_structured_output(schema=Person)

In [52]:
text= 'Alan Smith is 6 feet tall and has blond hair'
prompt = prompt_template.invoke({'text': text})
structured_model.invoke(prompt)

{'height': '6 feet', 'hair_color': 'blond'}